In [1]:
import os
import pydicom
import glob
import pandas as pd
from datetime import datetime


In [2]:
# load segmented cases  
prostate_lesion_seg = pd.read_csv("/data/oleksii/Prostate-Lesion-Datasets-NRRDS/lesion_dataset_1909_20240131_OB.csv", sep=";")

# load cases proposed by Lucas
cases_to_annotate = pd.read_excel("meta_data_lesion/20240702_Lucas_Cases_mit_KI_bewerten.xlsx")

# check for overlap
prostate_lesion_seg[prostate_lesion_seg["study_orthanc_id"].isin(cases_to_annotate["study_orthanc_id"])]

,study_orthanc_id,class,region,zone,segment_id,count,study_date,patient_id,csPCa,case_class,healthy,StudyInstanceUID,class_id


In [3]:
seq_map = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/sequence_mapping/sequence_mapping_13024_studies_20240115.csv", sep=";") 

In [4]:
# filter non-existing cases out
cases_to_annotate = cases_to_annotate[cases_to_annotate["study_orthanc_id"].isin(seq_map["study_orthanc_id"])]

dataset_annotate = seq_map[seq_map["study_orthanc_id"].isin(cases_to_annotate["study_orthanc_id"])]
dataset_annotate_full = dataset_annotate[~(dataset_annotate['t2w_tra_id'].isna() | dataset_annotate['adc_tra_id'].isna() | dataset_annotate['dwi_tra_id'].isna())]

In [6]:
# remove revently segmented cases from the list
lesion_studies_processed = os.listdir("/data/oleksii/alta-ai.com/alta-ai-orthanc-backup/2024-06-30/prostate_lesion/ProcessingState.PROCESSED")
dataset_annotate_full = dataset_annotate_full[~dataset_annotate_full["study_orthanc_id"].isin(lesion_studies_processed)]

In [13]:
# sent to Lucas for double checking
dataset_annotate_incomplete_seq = dataset_annotate[~dataset_annotate["study_orthanc_id"].isin(dataset_annotate_full["study_orthanc_id"])]
dataset_annotate_incomplete_seq.to_csv("annotate_incomplete_seq_map.csv", sep=";")


In [10]:
class_list = pd.read_csv("/home/oleksii/projects/ohif-orthanc-postgres-docker/datasets/classification/klassenliste_matched_with_pacs_20240411.csv", sep=";")
dataset_annotate_class = dataset_annotate_full[~dataset_annotate_full["study_orthanc_id"].isin(class_list["study_orthanc_id"])]
dataset_annotate_class.to_csv("annotate_batch_01_829_seq_map.csv", sep=";")


In [16]:
cases_to_annotate_first_batch = cases_to_annotate[cases_to_annotate["study_orthanc_id"].isin(dataset_annotate_class["study_orthanc_id"])]
cases_to_annotate_first_batch.to_csv("annotate_batch_01_829.csv", sep=";")
